# Homework 4: DFL/Massey Ratings
Amanda Kuznecov (anr431)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import poisson
import random
import scipy.stats as st
from scipy.special import logit, expit
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import brier_score_loss
from tqdm import tqdm
pd.set_option('display.max_columns', None)
import itertools

In [2]:
#read in data
df = pd.read_csv('hw4.csv', index_col = False, 
                 parse_dates = ['Date'])

#sort by date to update ratings chronologically
df = df.sort_values('Date')

## Question 1
### Part a

Something about taking previous season's results and penalizing slightly so that we predict good goal differentials but not quite as good as the previous year. Using this penalty, the high outliers come down and the low outliers move up.

### Part b

In [3]:
train = df.loc[(df.Y >= 14) & (df.Y < 18)]

In [4]:
#checking coeff for goal differential model
goal_diff_model = smf.glm('I(GD_Home) ~ I(logit(pH)-logit(pA))-1', data = train).fit()
goal_diff_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             I(GD_Home)   No. Observations:                 7304
Model:                            GLM   Df Residuals:                     7303
Model Family:                Gaussian   Df Model:                            0
Link Function:               identity   Scale:                          2.5139
Method:                          IRLS   Log-Likelihood:                -13730.
Date:                Sat, 06 Mar 2021   Deviance:                       18359.
Time:                        20:20:02   Pearson chi2:                 1.84e+04
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
I(logit(pH) - logit(pA))     0.4897      0.009     53.405      0.000       0.472       0.508
============================================================================================
"""

#### Part i

In [5]:
#model for converting shot differentials to goal differentials
shot_diff_model = smf.glm('GD_Home ~ SD_Home -1', data = train).fit()
shot_diff_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                GD_Home   No. Observations:                 7304
Model:                            GLM   Df Residuals:                     7303
Model Family:                Gaussian   Df Model:                            0
Link Function:               identity   Scale:                          3.0262
Method:                          IRLS   Log-Likelihood:                -14407.
Date:                Sat, 06 Mar 2021   Deviance:                       22101.
Time:                        20:20:02   Pearson chi2:                 2.21e+04
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
SD_Home        0.0819      0.002     33.659      0.000       0.077       0.087
==============================================================================
"""

#### Part ii

In [6]:
#model for converting shot differentials to goal differentials
shot_diff_model = smf.glm('GD_Home ~ xGD_Home-1', data = train).fit()
shot_diff_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                GD_Home   No. Observations:                 7304
Model:                            GLM   Df Residuals:                     7303
Model Family:                Gaussian   Df Model:                            0
Link Function:               identity   Scale:                          1.7748
Method:                          IRLS   Log-Likelihood:                -12459.
Date:                Sat, 06 Mar 2021   Deviance:                       12962.
Time:                        20:20:02   Pearson chi2:                 1.30e+04
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
xGD_Home       0.9938      0.012     84.147      0.000       0.971       1.017
==============================================================================
"""

### Part c

In [7]:
#import team priors
df_prior = pd.read_csv('hw4_prior.csv')

In [8]:
hfa_prior = 0.3739 #Prior on home field advantage for goal differential
lmp_to_goal = 0.489739 #Conversion factor from differences of logit market probs to goals
team_prior_wt = 15 #Weight on team priors from previous season
hfa_prior_wt = 500 #Weight on hfa prior (strong)
wt_decay = 0.97 #Decay of weights per day, approx 0.81 per week
mkt_wt = 10 #Weight of market prices
goal_wt = 1 #Weight of goal differentials

#added
shot_to_goal = 0.0819 #conversion factor from shot differentials to goal differentials
xg_to_goal = 0.9938 #conversion factor from expected goal differentials to goal differentials
shot_wt = 1 #weight of shot differentials
xg_wt = 1 #weight of expected goal differentials

In [9]:
df_post = pd.DataFrame(columns = ['Div','Y','Team','Post_Rating'])
data = []
for div in tqdm(sorted(df.Div.unique())) : #Process each division
    for y in sorted(df.loc[df.Y.between(15,18)].Y.unique()) : #Process each year separately
        df_dy = df.loc[(df.Y == y) & (df.Div == div)].copy() #contains only data for that year and div
        n_games = len(df_dy) #how many games played that season for that div
        teams = sorted(set(df_dy.Team_Home.unique()) | set(df_dy.Team_Away.unique())) #set of distinct teams
        n_teams = len(teams) #number of teams
        team_map = {k:i for i,k in enumerate(teams)} #maps from team name -> index
        
        n_rows = 4*n_games + n_teams + 1 #n_teams team priors, 1 hfa prior, 4 rows per game #CHANGED
        X = np.zeros((n_rows, n_teams + 1)) 
        Y = np.zeros(n_rows) 
        wgts = np.ones(n_rows) #by default, set all weights to 1 #CHANGED
        
        #Setup priors
        X[:n_teams+1,:n_teams+1] = np.eye(n_teams+1) #rows for priors
        df_prior_dy = df_prior.loc[(df_prior.Y == y) & (df_prior.Div == div)].copy()
        #Setup team priors
        wgts[:n_teams] = team_prior_wt
        for i in range(len(df_prior_dy)) :
            team = df_prior_dy.Team.iloc[i]
            prior = df_prior_dy.priorGD.iloc[i]
            Y[team_map[team]] = prior
        #Setup hfa prior
        Y[n_teams] = hfa_prior
        wgts[n_teams] = hfa_prior_wt
        
        #Game rows are an alternating pattern of goal differentials, shot diff, xg diff, mkt_price
        wgts[n_teams+1::4] = goal_wt #added
        wgts[n_teams+2::4] = mkt_wt #CHANGED
        wgts[n_teams+3::4] = shot_wt #added
        wgts[n_teams+4::4] = xg_wt #added
        
        #Starting ratings are just priors
        ratings = Y[:n_teams]

        prev_date = None
        ratings_home = np.empty(n_games)
        ratings_away = np.empty(n_games)
        
        #Process every game
        for i in range(n_games) : #dataframe is sorted chronologically
            curr_date = df_dy.Date.iloc[i]
            #Refit on all strictly earlier games if first game of new date
            if prev_date is not None and curr_date > prev_date :
                rating_model = sm.WLS(Y, X, weights = wgts).fit()
                ratings = rating_model.params[:n_teams]
                ratings = ratings - np.mean(ratings) #Center ratings around 0            
                #Decay weights of all games and priors using elapsed days
                wgts[:n_teams+1+4*i] *= wt_decay**((curr_date-prev_date)/pd.Timedelta(1,unit='day')) #CHANGED
            prev_date = curr_date
            
            home, away = df_dy.Team_Home.iloc[i], df_dy.Team_Away.iloc[i]
            i_home, i_away = team_map[home], team_map[away]
            ratings_home[i] = ratings[i_home]
            ratings_away[i] = ratings[i_away]
            
            X[n_teams+1+4*i:n_teams+1+4*i+4] = 1.0*(np.arange(n_teams+1)==i_home)-1.0*(np.arange(n_teams+1)==i_away) #CHANGED
            X[n_teams+1+4*i:n_teams+1+4*i+4, -1] = 1.0 #HFA #CHANGED 
            Y[n_teams+1+4*i] = df_dy.GD_Home.iloc[i] #Goal differential #CHANGED
            Y[n_teams+1+4*i+1] = lmp_to_goal * (logit(df_dy.pH.iloc[i])-logit(df_dy.pA.iloc[i])) #market info #CHANGED
            Y[n_teams+1+4*i+2] = shot_to_goal * df_dy.SD_Home.iloc[i] #added
            Y[n_teams+1+4*i+3] = xg_to_goal * df_dy.xGD_Home.iloc[i] #added
            
        df_dy['R_Home'] = ratings_home
        df_dy['R_Away'] = ratings_away
        data.append(df_dy)
        
        #calculate post season ratings based on last weight update
        postseason_model = sm.WLS(Y, X, weights = wgts).fit()
        post_ratings = postseason_model.params[:n_teams]
        post_ratings = post_ratings - np.mean(post_ratings) 
        df_scratch = pd.DataFrame({'Div': div,'Y':y,'Team': teams,'Post_Rating': post_ratings}) #create df for post season ratings
        df_post = df_post.append(df_scratch) #append to full df of post season ratings
        

df_ratings = pd.concat(data).sort_values('GameID')
df_ratings.head()

100%|██████████| 5/5 [00:08<00:00,  1.63s/it]


,Div,Date,Y,Team_Home,Team_Away,G_Home,G_Away,S_Home,S_Away,pH,pD,pA,xG_Home,xG_Away,GameID,GD_Home,GD_Away,SD_Home,SD_Away,xGD_Home,xGD_Away,R_Home,R_Away
1826,Ligue_1,2015-08-07,15,Lille,Paris SG,0,1,12,7,0.147474,0.259055,0.593471,0.914879,1.334570,1826,-1,1,5,-5,-0.419691,0.419691,0.005618,1.066845
1827,EPL,2015-08-08,15,Everton,Watford,2,2,10,11,0.577077,0.248362,0.174561,0.604226,0.557892,1827,0,0,-1,1,0.046334,-0.046334,-0.056671,-0.509390
1828,EPL,2015-08-08,15,Bournemouth,Aston Villa,0,1,11,7,0.502280,0.268341,0.229378,0.876106,0.782253,1828,-1,1,4,-4,0.093853,-0.093853,-0.509390,-0.610355
1829,Ligue_1,2015-08-08,15,Nice,Monaco,1,2,5,19,0.235607,0.288583,0.475810,0.459874,2.814130,1829,-1,1,-14,14,-2.354256,2.354256,-0.225083,0.559302
1830,Ligue_1,2015-08-08,15,Troyes,Ajaccio GFCO,0,0,12,6,0.480552,0.299240,0.220208,0.394458,1.059770,1830,0,0,6,-6,-0.665312,0.665312,-0.516311,-0.516311


In [10]:
#df for pre-season ratings for all teams in EPL in 2017 season 
epl_prior17 = df_prior.loc[(df_prior.Div == 'EPL')& (df_prior.Y == 17)]

#df for post-season ratings for all teams in EPL in 2017 season
epl_post17 = df_post.loc[(df_post.Div == 'EPL')& (df_post.Y == 17)]

#df containing post & pre-season ratings
epl17 = epl_prior17.merge(epl_post17)
epl17 = epl17.sort_values('Post_Rating',ascending = False)
epl17 = epl17.drop(columns = ['Div','Y'])

In [11]:
epl17

,Team,priorGD,Post_Rating
10,Man City,0.894973,1.452859
9,Liverpool,0.779622,1.002248
16,Tottenham,1.333306,0.940018
4,Chelsea,1.148744,0.794985
11,Man United,0.525850,0.712594
0,Arsenal,0.710411,0.435926
13,Southampton,-0.212395,-0.035730
5,Crystal Palace,-0.350815,-0.061418
8,Leicester,-0.396956,-0.229420
12,Newcastle,-0.549762,-0.249825


### Part d

In [12]:
#build model for home wins based on rating differential
df_ratings['HomeWin'] = 1.0*(df_ratings.G_Home > df_ratings.G_Away)
df57 = df_ratings.loc[df_ratings.Y.between(15,17)]
df8 = df_ratings.loc[df_ratings.Y == 18]
logit_model = smf.logit('HomeWin ~ I(R_Home-R_Away)', df57).fit()
print(logit_model.summary())
print('logit:',brier_score_loss(df8.HomeWin, logit_model.predict(df8)))

Optimization terminated successfully.
         Current function value: 0.610079
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                HomeWin   No. Observations:                 5478
Model:                          Logit   Df Residuals:                     5476
Method:                           MLE   Df Model:                            1
Date:                Sat, 06 Mar 2021   Pseudo R-squ.:                  0.1160
Time:                        20:20:14   Log-Likelihood:                -3342.0
converged:                       True   LL-Null:                       -3780.6
Covariance Type:            nonrobust   LLR p-value:                8.687e-193
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.1823      0.029     -6.194      0.000      -0.240      -0.125
I(R_H

### Part e

In [13]:
#build model for pH based on ratings differential
logit_model = smf.logit('I(pH)~ I(R_Home-R_Away)', df57).fit()
print(logit_model.summary())
print('logit:',brier_score_loss(df8.HomeWin, logit_model.predict(df8)))

Optimization terminated successfully.
         Current function value: 0.555208
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                  I(pH)   No. Observations:                 5478
Model:                          Logit   Df Residuals:                     5476
Method:                           MLE   Df Model:                            1
Date:                Sat, 06 Mar 2021   Pseudo R-squ.:                  0.1640
Time:                        20:20:17   Log-Likelihood:                -3041.4
converged:                       True   LL-Null:                       -3638.1
Covariance Type:            nonrobust   LLR p-value:                1.649e-261
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.2217      0.029     -7.537      0.000      -0.279      -0.164
I(R_H

### Part f.i & ii

In [14]:
def ratings(df, df_prior, mkt_wt, goal_wt, shot_wt, xg_wt, start_yr, end_yr):
    
    #priors and weights
    hfa_prior = 0.3739 #Prior on home field advantage for goal differential
    lmp_to_goal = 0.489739 #Conversion factor from differences of logit market probs to goals
    team_prior_wt = 15 #Weight on team priors from previous season
    hfa_prior_wt = 500 #Weight on hfa prior (strong)
    wt_decay = 0.97 #Decay of weights per day, approx 0.81 per week
    shot_to_goal = 0.0819
    xg_to_goal = 0.9938

    data = []
    for div in sorted(df.Div.unique()) : #Process each division
        for y in sorted(df.loc[df.Y.between(start_yr,end_yr)].Y.unique()) : #Process each year separately
            df_dy = df.loc[(df.Y == y) & (df.Div == div)].copy()
            n_games = len(df_dy)
            teams = sorted(set(df_dy.Team_Home.unique()) | set(df_dy.Team_Away.unique())) #All teams
            n_teams = len(teams)
            team_map = {k:i for i,k in enumerate(teams)} #team -> index

            n_rows = 4*n_games + n_teams + 1 #n_teams team priors, 1 hfa prior, 4 rows per game #CHANGED
            X = np.zeros((n_rows, n_teams + 1)) 
            Y = np.zeros(n_rows) 
            wgts = np.ones(n_rows) #by default, set all weights to 1 #CHANGED

            #Setup priors
            X[:n_teams+1,:n_teams+1] = np.eye(n_teams+1) #rows for priors
            df_prior_dy = df_prior.loc[(df_prior.Y == y) & (df_prior.Div == div)].copy()
            #Setup team priors
            wgts[:n_teams] = team_prior_wt
            for i in range(len(df_prior_dy)) :
                team = df_prior_dy.Team.iloc[i]
                prior = df_prior_dy.priorGD.iloc[i]
                Y[team_map[team]] = prior
            #Setup hfa prior
            Y[n_teams] = hfa_prior
            wgts[n_teams] = hfa_prior_wt

            #Game rows are an alternating pattern of goal differentials, shot diff, xg diff, mkt_price
            wgts[n_teams+1::4] = goal_wt #added
            wgts[n_teams+2::4] = mkt_wt #CHANGED
            wgts[n_teams+3::4] = shot_wt #added
            wgts[n_teams+4::4] = xg_wt #added

            #Starting ratings are just priors
            ratings = Y[:n_teams]

            prev_date = None
            ratings_home = np.empty(n_games)
            ratings_away = np.empty(n_games)

            #Process every game
            for i in range(n_games) : #dataframe is sorted chronologically
                curr_date = df_dy.Date.iloc[i]
                #Refit on all strictly earlier games if first game of new date
                if prev_date is not None and curr_date > prev_date :
                    rating_model = sm.WLS(Y, X, weights = wgts).fit()
                    ratings = rating_model.params[:n_teams]
                    ratings = ratings - np.mean(ratings) #Center ratings around 0            
                    #Decay weights of all games and priors using elapsed days
                    wgts[:n_teams+1+4*i] *= wt_decay**((curr_date-prev_date)/pd.Timedelta(1,unit='day')) #CHANGED
                prev_date = curr_date

                home, away = df_dy.Team_Home.iloc[i], df_dy.Team_Away.iloc[i]
                i_home, i_away = team_map[home], team_map[away]
                ratings_home[i] = ratings[i_home]
                ratings_away[i] = ratings[i_away]

                X[n_teams+1+4*i:n_teams+1+4*i+4] = 1.0*(np.arange(n_teams+1)==i_home)-1.0*(np.arange(n_teams+1)==i_away) #CHANGED
                X[n_teams+1+4*i:n_teams+1+4*i+4, -1] = 1.0 #HFA #CHANGED 
                Y[n_teams+1+4*i] = df_dy.GD_Home.iloc[i] #Goal differential #CHANGED
                Y[n_teams+1+4*i+1] = lmp_to_goal * (logit(df_dy.pH.iloc[i])-logit(df_dy.pA.iloc[i])) #market info #CHANGED
                Y[n_teams+1+4*i+2] = shot_to_goal * df_dy.SD_Home.iloc[i] #added
                Y[n_teams+1+4*i+3] = xg_to_goal * df_dy.xGD_Home.iloc[i] #added

            df_dy['R_Home'] = ratings_home
            df_dy['R_Away'] = ratings_away
            data.append(df_dy)

    df_ratings = pd.concat(data).sort_values('GameID')

    return df_ratings

### Part f.iii

In [57]:
#tuning weights
mkt_wt = [10,20,30,40,60]
goal_wt = [1,2,3,4,5]
shot_wt = [1,2,3]
xg_wt = [1,2,3]

In [58]:
wt_combs = list(itertools.product(mkt_wt, goal_wt, shot_wt, xg_wt))

In [59]:
ls = []
#iterate over possible combinations of weights and calc brier scores
for i in tqdm(wt_combs):
    mkt_wt, goal_wt, shot_wt, xg_wt = i
    df_ratings = ratings(df, df_prior, mkt_wt, goal_wt, shot_wt, xg_wt, 15, 18)
    df_ratings['HomeWin'] = 1.0*(df_ratings.G_Home > df_ratings.G_Away)

    #train model on 2015-2017 data
    df57 = df_ratings.loc[df_ratings.Y.between(15,17)]
    df8 = df_ratings.loc[df_ratings.Y == 18]
    logit_model = smf.logit('I(pH)~ I(R_Home-R_Away)', df57).fit()
    
    ls.append([mkt_wt, goal_wt, shot_wt, xg_wt,brier_score_loss(df8.HomeWin, logit_model.predict(df8))])

  0%|          | 1/225 [00:07<27:16,  7.31s/it]

Optimization terminated successfully.
         Current function value: 0.555208
         Iterations 5


  1%|          | 2/225 [00:14<27:28,  7.39s/it]

Optimization terminated successfully.
         Current function value: 0.555532
         Iterations 5


  1%|▏         | 3/225 [00:21<26:33,  7.18s/it]

...

Optimization terminated successfully.
         Current function value: 0.555182
         Iterations 5


100%|█████████▉| 224/225 [26:56<00:07,  7.27s/it]

Optimization terminated successfully.
         Current function value: 0.555133
         Iterations 5


100%|██████████| 225/225 [27:02<00:00,  7.21s/it]

Optimization terminated successfully.
         Current function value: 0.555103
         Iterations 5


In [60]:
#find lowest brier and corresponding params
br = 1
for i in range(len(ls)):
    if ls[i][4]< br:
        mkt_wt, goal_wt, shot_wt, xg_wt, br = ls[i]

In [61]:
print('Tuned parameters:')
print(f'mkt_wt:{mkt_wt}, goal_wt: {goal_wt}, shot_wt: {shot_wt}, xg_wt: {xg_wt}')
print(f'Brier score: {br}')

Tuned parameters:
mkt_wt:40, goal_wt: 3, shot_wt: 3, xg_wt: 3
Brier score: 0.21135674433209028


### Part f.iv

### 2015-2017 Season Model Building & Newly Promoted Team Priors

###### Newly Promoted Teams

In [15]:
#create df for goal differentials per team
df_gd = df[['GameID','Y','Team_Home','Team_Away','GD_Home','GD_Away']].copy()
df_long = pd.wide_to_long(df_gd, ['Team','GD'], i = ['GameID','Y'], 
                          j = 'isHome', sep = '_', suffix = r'\w+')
df_long = df_long.reset_index().sort_values(['GameID','Y']) 

In [16]:
df_long.head()

,GameID,Y,isHome,Team,GD
0,0,14,Home,Reims,0
1,0,14,Away,Paris SG,0
2,1,14,Home,Montpellier,-1
3,1,14,Away,Bordeaux,1
4,2,14,Home,Lille,0


In [17]:
#get all newly promoted teams during 2015-2017
years = [15,16,17]
teams = df_long.Team.unique()
newly_promoted = []
for year in years:
    for team in teams:
        if (len(df_long.loc[(df_long.Team == team)&(df_long.Y == (year))])>0 and len(df_long.loc[(df_long.Team == team)&(df_long.Y == (year-1))])==0):
            newly_promoted.append([team, year])

In [18]:
#get avg GD avg of newly promoted teams during 2015-2017
num_games = 0
GD_sum = 0
GDavg = []
for team, year in newly_promoted:
    df_filt = df_long.loc[(df_long.Team == team)&(df_long.Y == year)]
    #num_games += len(df_filt)
    #GD_sum += df_filt.GD.sum()
    GDavg.append(df_filt.GD.mean())
#GDavg_newpromo2 = GD_sum/num_games
GDavg_newpromo = sum(GDavg)/len(GDavg)

In [19]:
#use this as prior on teams that were newly promoted
GDavg_newpromo

-0.5325445967860828

##### Teams already in league

In [20]:
#create df for goal differentials per team
df_gd = df[['GameID','Y','Div','Team_Home','Team_Away','GD_Home','GD_Away']].copy()

#only 2014-2016 data used as GD_prev, 2015-2017 data used as GD
df_gd = df_gd.loc[df_gd.Y < 18]
df_long = pd.wide_to_long(df_gd, ['Team','GD'], i = ['GameID','Div','Y'], 
                          j = 'isHome', sep = '_', suffix = r'\w+')
df_long = df_long.reset_index().sort_values(['GameID','Y']) 

In [21]:
GD_savg = df_long.groupby(['Div','Team', 'Y'])['GD'].mean().reset_index()

#column for GD prev based on previous year's GD avg for a team
GD_savg['GD_prev'] = GD_savg.groupby(('Team'))['GD'].transform(lambda x: x.shift(1,fill_value = 0))

#model using 2015-2017 data with prior on each year (2014-2016)
GD_savg = GD_savg.loc[GD_savg.Y > 14]

In [22]:
GD_savg.head()

,Div,Team,Y,GD,GD_prev
1,Bundesliga,Augsburg,15,-0.294118,0.000000
2,Bundesliga,Augsburg,16,-0.470588,-0.294118
3,Bundesliga,Augsburg,17,-0.088235,-0.470588
5,Bundesliga,Bayern Munich,15,1.852941,1.823529
6,Bundesliga,Bayern Munich,16,1.970588,1.852941


In [23]:
#remove teams that have just been promoted that year
ind_ls = []
for team, year in newly_promoted:
    ind_ls.append(GD_savg.loc[(GD_savg.Team == team)& (GD_savg.Y == year)].index.item())
GD_savg_nonpromo = GD_savg.loc[~GD_savg.index.isin(ind_ls)]

In [24]:
GD_savg_nonpromo.head()

,Div,Team,Y,GD,GD_prev
1,Bundesliga,Augsburg,15,-0.294118,0.000000
2,Bundesliga,Augsburg,16,-0.470588,-0.294118
3,Bundesliga,Augsburg,17,-0.088235,-0.470588
5,Bundesliga,Bayern Munich,15,1.852941,1.823529
6,Bundesliga,Bayern Munich,16,1.970588,1.852941


In [25]:
goal_diff_model = smf.glm('GD ~ GD_prev', data = GD_savg_nonpromo).fit()
goal_diff_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                     GD   No. Observations:                  252
Model:                            GLM   Df Residuals:                      250
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                         0.20309
Method:                          IRLS   Log-Likelihood:                -155.71
Date:                Sat, 06 Mar 2021   Deviance:                       50.772
Time:                        20:20:43   Pearson chi2:                     50.8
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0337      0.029     -1.162      0.245      -0.090       0.023
GD_prev        0.8767      0.042     20.915      0.000       0.795       0.959
==============================================================================
"""

In [26]:
#get scaling factors from regression
intercept, coeff = goal_diff_model.params

###### Set priors

In [27]:
#apply conversion to calculate priors on teams who were already in the league
GD_savg_nonpromo = GD_savg_nonpromo.copy() #to avoid set copy issue
GD_savg_nonpromo['prior'] = GD_savg_nonpromo['GD_prev']*coeff+intercept
GD_savg_nonpromo.head()

,Div,Team,Y,GD,GD_prev,prior
1,Bundesliga,Augsburg,15,-0.294118,0.000000,-0.033686
2,Bundesliga,Augsburg,16,-0.470588,-0.294118,-0.291528
3,Bundesliga,Augsburg,17,-0.088235,-0.470588,-0.446234
5,Bundesliga,Bayern Munich,15,1.852941,1.823529,1.564940
6,Bundesliga,Bayern Munich,16,1.970588,1.852941,1.590724


In [28]:
#set prior on all newly promoted teams to avg GDavg for newly promoted teams during 2015-2017
GD_savg_newpromo = GD_savg.loc[GD_savg.index.isin(ind_ls)].copy()
GD_savg_newpromo['prior'] = GDavg_newpromo

In [29]:
GD_savg_newpromo.head()

,Div,Team,Y,GD,GD_prev,prior
8,Bundesliga,Darmstadt,15,-0.441176,0.000000,-0.532545
23,Bundesliga,Freiburg,16,-0.529412,-0.323529,-0.532545
31,Bundesliga,Hannover,17,-0.294118,-0.911765,-0.532545
40,Bundesliga,Ingolstadt,15,-0.264706,0.000000,-0.532545
55,Bundesliga,RB Leipzig,16,0.794118,0.000000,-0.532545


###### Combine data for all teams

In [30]:
#concat dfwith newly promoted teams and df with teams who were already in the league
all_GD_savg = pd.concat([GD_savg_nonpromo, GD_savg_newpromo])
all_GD_savg.head()

,Div,Team,Y,GD,GD_prev,prior
1,Bundesliga,Augsburg,15,-0.294118,0.000000,-0.033686
2,Bundesliga,Augsburg,16,-0.470588,-0.294118,-0.291528
3,Bundesliga,Augsburg,17,-0.088235,-0.470588,-0.446234
5,Bundesliga,Bayern Munich,15,1.852941,1.823529,1.564940
6,Bundesliga,Bayern Munich,16,1.970588,1.852941,1.590724


In [31]:
#get seasonal GD per division for normalizing
p_mean = pd.DataFrame(all_GD_savg.groupby(['Div','Y'])['prior'].mean())
p_mean = p_mean.reset_index()
p_mean = p_mean.rename(columns= {'prior': 'p_mean'})

#include normalizing value in df
all_GD_savg = all_GD_savg.merge(p_mean)

#normalize priors
all_GD_savg['prior_norm'] = all_GD_savg['prior']-all_GD_savg['p_mean']

In [32]:
#check prior_norm vs df_prior below
all_GD_savg.loc[all_GD_savg.Team == 'Arsenal']

,Div,Team,Y,GD,GD_prev,prior,p_mean,prior_norm
54,EPL,Arsenal,15,0.763158,0.921053,0.773770,-0.023155,0.796925
74,EPL,Arsenal,16,0.868421,0.763158,0.635349,0.004529,0.630819
94,EPL,Arsenal,17,0.605263,0.868421,0.727629,0.017218,0.710411


In [33]:
df_prior.loc[df_prior.Team == 'Arsenal']

,Div,Y,Team,priorGD
72,EPL,15,Arsenal,0.796925
92,EPL,16,Arsenal,0.630819
112,EPL,17,Arsenal,0.710411
132,EPL,18,Arsenal,0.506241


###  Function to Create Priors on Next Seasons

In [34]:
def create_priors(df, season):
    #create df for goal differentials per team
    df_gd = df[['GameID','Y','Div','Team_Home','Team_Away','GD_Home','GD_Away']].copy()
    df_long = pd.wide_to_long(df_gd, ['Team','GD'], i = ['GameID','Div','Y'], 
                              j = 'isHome', sep = '_', suffix = r'\w+')
    df_long = df_long.reset_index().sort_values(['GameID','Y']) 
    
    GD_savg = df_long.groupby(['Div','Team', 'Y'])['GD'].mean().reset_index()

    #column for GD prev based on previous year's GD avg for a team
    GD_savg['GD_prev'] = GD_savg.groupby(('Team'))['GD'].transform(lambda x: x.shift(1,fill_value = 0))
    
    #get all newly promoted teams during 2015-2017
    teams = df_long.Team.unique()
    newly_promoted = []
    for team in teams:
        if (len(df_long.loc[(df_long.Team == team)&(df_long.Y == (season))])>0 and len(df_long.loc[(df_long.Team == team)&(df_long.Y == (season-1))])==0):
            newly_promoted.append([team, season])
            
    df_gd = df_gd.loc[(df_gd.Y > (season-2)) & (df_gd.Y < (season+1))]
    
    GD_savg = df_long.groupby(['Div','Team', 'Y'])['GD'].mean().reset_index()

    #column for GD prev based on previous year's GD avg for a team
    GD_savg['GD_prev'] = GD_savg.groupby(('Team'))['GD'].transform(lambda x: x.shift(1,fill_value = 0))

    #set to current season
    GD_savg = GD_savg.loc[GD_savg.Y == season]
    
    #get indices of teams that have just been promoted in that season
    ind_ls = []
    for team, year in newly_promoted:
        ind_ls.append(GD_savg.loc[(GD_savg.Team == team)& (GD_savg.Y == year)].index.item())
    GD_savg_nonpromo = GD_savg.loc[~GD_savg.index.isin(ind_ls)]
    #set prior on all newly promoted teams to avg GDavg for newly promoted teams during 2015-2017
    GD_savg_newpromo = GD_savg.loc[GD_savg.index.isin(ind_ls)].copy()
    GD_savg_newpromo['prior'] = GDavg_newpromo

    #apply conversion to calculate priors on teams who were already in the league
    GD_savg_nonpromo = GD_savg_nonpromo.copy() #to avoid set copy issue
    GD_savg_nonpromo['prior'] = GD_savg_nonpromo['GD_prev']*coeff+intercept
    GD_savg_nonpromo.head()

    #concat df with newly promoted teams and df with teams who were already in the league
    all_GD_savg = pd.concat([GD_savg_nonpromo, GD_savg_newpromo])
    all_GD_savg.head()

    #get seasonal GD per division for normalizing
    p_mean = pd.DataFrame(all_GD_savg.groupby(['Div','Y'])['prior'].mean())
    p_mean = p_mean.reset_index()
    p_mean = p_mean.rename(columns= {'prior': 'p_mean'})

    #include normalizing value in df
    all_GD_savg = all_GD_savg.merge(p_mean)

    #normalize priors
    all_GD_savg['priorGD'] = all_GD_savg['prior']-all_GD_savg['p_mean']
    
    #remove other columns
    all_GD_savg = all_GD_savg.drop(columns = ['GD','GD_prev','prior','p_mean'])
    
    return all_GD_savg

In [35]:
#check against Brett's
df_prior18 = create_priors(df,18)
df_prior18.head()

,Div,Team,Y,priorGD
0,Bundesliga,Augsburg,18,-0.106439
1,Bundesliga,Bayern Munich,18,1.621108
2,Bundesliga,Dortmund,18,0.409246
3,Bundesliga,Ein Frankfurt,18,-0.029086
4,Bundesliga,Freiburg,18,-0.647909


In [36]:
df_prior.loc[df_prior.Y == 18].head()

,Div,Y,Team,priorGD
54,Bundesliga,18,Augsburg,-0.106439
55,Bundesliga,18,Bayern Munich,1.621108
56,Bundesliga,18,Dortmund,0.409246
57,Bundesliga,18,Ein Frankfurt,-0.029086
58,Bundesliga,18,Fortuna Dusseldorf,-0.527945


In [37]:
#create priors on 2019
df_prior19 = create_priors(df,19)

#concat with other priors
df_prior = pd.concat([df_prior, df_prior19])

In [47]:
#run ratings up to 2019 on tuned model
mkt_wt=40
goal_wt=3 
shot_wt=3
xg_wt=3
df_ratings = ratings(df, df_prior, mkt_wt, goal_wt, shot_wt, xg_wt, 15, 19)
df_ratings19 = df_ratings.loc[df_ratings.Y == 19].copy()
df_ratings19['HomeWin'] = 1.0*(df_ratings19.G_Home > df_ratings19.G_Away)

In [48]:
#only include precovid games
df_precovid = df_ratings19.loc[df_ratings19.Date < '2020-03-12']

In [49]:
#get Brier score of tuned model on 2019 data
print('logit:',brier_score_loss(df_precovid.HomeWin, logit_model.predict(df_precovid)))

logit: 0.22132407332161363


### Part g

In [50]:
#create priors on 2020
df_prior20 = create_priors(df,20)

#concat with other priors
df_prior = pd.concat([df_prior, df_prior20])

#### Part g.ii (using Original HFA prior)

In [54]:
#run ratings up to 2020 on tuned model
df_ratings = ratings(df, df_prior, mkt_wt, goal_wt, shot_wt, xg_wt, 15, 20)
df_ratings20 = df_ratings.loc[df_ratings.Y == 20].copy()
df_ratings20['HomeWin'] = 1.0*(df_ratings20.G_Home > df_ratings20.G_Away)

In [55]:
#get Brier score of tuned model on 2020 data
print('logit:',brier_score_loss(df_ratings20.HomeWin, logit_model.predict(df_ratings20)))

logit: 0.2078170706034281


#### Part g.iii (using covid HFA prior)

In [61]:
def ratings_covid(df, df_prior, mkt_wt, goal_wt, shot_wt, xg_wt, start_yr, end_yr):
    
    #priors and weights
    hfa_prior = 0.3739/3 #Prior on home field advantage for goal differential
    lmp_to_goal = 0.489739 #Conversion factor from differences of logit market probs to goals
    team_prior_wt = 15 #Weight on team priors from previous season
    hfa_prior_wt = 500 #Weight on hfa prior (strong)
    wt_decay = 0.97 #Decay of weights per day, approx 0.81 per week
    shot_to_goal = 0.0819
    xg_to_goal = 0.9938

    data = []
    for div in sorted(df.Div.unique()) : #Process each division
        for y in sorted(df.loc[df.Y.between(start_yr,end_yr)].Y.unique()) : #Process each year separately
            df_dy = df.loc[(df.Y == y) & (df.Div == div)].copy()
            n_games = len(df_dy)
            teams = sorted(set(df_dy.Team_Home.unique()) | set(df_dy.Team_Away.unique())) #All teams
            n_teams = len(teams)
            team_map = {k:i for i,k in enumerate(teams)} #team -> index

            n_rows = 4*n_games + n_teams + 1 #n_teams team priors, 1 hfa prior, 4 rows per game #CHANGED
            X = np.zeros((n_rows, n_teams + 1)) 
            Y = np.zeros(n_rows) 
            wgts = np.ones(n_rows) #by default, set all weights to 1 #CHANGED

            #Setup priors
            X[:n_teams+1,:n_teams+1] = np.eye(n_teams+1) #rows for priors
            df_prior_dy = df_prior.loc[(df_prior.Y == y) & (df_prior.Div == div)].copy()
            #Setup team priors
            wgts[:n_teams] = team_prior_wt
            for i in range(len(df_prior_dy)) :
                team = df_prior_dy.Team.iloc[i]
                prior = df_prior_dy.priorGD.iloc[i]
                Y[team_map[team]] = prior
            #Setup hfa prior
            Y[n_teams] = hfa_prior
            wgts[n_teams] = hfa_prior_wt

            #Game rows are an alternating pattern of goal differentials, shot diff, xg diff, mkt_price
            wgts[n_teams+1::4] = goal_wt #added
            wgts[n_teams+2::4] = mkt_wt #CHANGED
            wgts[n_teams+3::4] = shot_wt #added
            wgts[n_teams+4::4] = xg_wt #added

            #Starting ratings are just priors
            ratings = Y[:n_teams]

            prev_date = None
            ratings_home = np.empty(n_games)
            ratings_away = np.empty(n_games)

            #Process every game
            for i in range(n_games) : #dataframe is sorted chronologically
                curr_date = df_dy.Date.iloc[i]
                #Refit on all strictly earlier games if first game of new date
                if prev_date is not None and curr_date > prev_date :
                    rating_model = sm.WLS(Y, X, weights = wgts).fit()
                    ratings = rating_model.params[:n_teams]
                    ratings = ratings - np.mean(ratings) #Center ratings around 0            
                    #Decay weights of all games and priors using elapsed days
                    wgts[:n_teams+1+4*i] *= wt_decay**((curr_date-prev_date)/pd.Timedelta(1,unit='day')) #CHANGED
                prev_date = curr_date

                home, away = df_dy.Team_Home.iloc[i], df_dy.Team_Away.iloc[i]
                i_home, i_away = team_map[home], team_map[away]
                ratings_home[i] = ratings[i_home]
                ratings_away[i] = ratings[i_away]

                X[n_teams+1+4*i:n_teams+1+4*i+4] = 1.0*(np.arange(n_teams+1)==i_home)-1.0*(np.arange(n_teams+1)==i_away) #CHANGED
                X[n_teams+1+4*i:n_teams+1+4*i+4, -1] = 1.0 #HFA #CHANGED 
                Y[n_teams+1+4*i] = df_dy.GD_Home.iloc[i] #Goal differential #CHANGED
                Y[n_teams+1+4*i+1] = lmp_to_goal * (logit(df_dy.pH.iloc[i])-logit(df_dy.pA.iloc[i])) #market info #CHANGED
                Y[n_teams+1+4*i+2] = shot_to_goal * df_dy.SD_Home.iloc[i] #added
                Y[n_teams+1+4*i+3] = xg_to_goal * df_dy.xGD_Home.iloc[i] #added

            df_dy['R_Home'] = ratings_home
            df_dy['R_Away'] = ratings_away
            data.append(df_dy)

    df_ratings = pd.concat(data).sort_values('GameID')

    return df_ratings

In [62]:
#run ratings up to 2020 on tuned model
df_ratings = ratings_covid(df, df_prior, mkt_wt, goal_wt, shot_wt, xg_wt, 15, 20)
df_ratings20cov = df_ratings.loc[df_ratings.Y == 20].copy()
df_ratings20cov['HomeWin'] = 1.0*(df_ratings20cov.G_Home > df_ratings20cov.G_Away)

In [63]:
#get Brier score of tuned model on 2020 data
print('logit:',brier_score_loss(df_ratings20cov.HomeWin, logit_model.predict(df_ratings20cov)))

logit: 0.2075781385088523
